## local Modeling 

start with 

```python
'Tokens_vectors_text/Data_for_modeling.parquet'
```

> this data has been processed in azure and contains the data and feature vectors ready to model


In [1]:
pwd

'/home/ucsddse230/work/final_project/post_azure'

In [2]:
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions  as psf
from pyspark.sql import Window 
from pyspark.sql.types  import DateType
import pandas as pd
#pd.set_option('display.max_colwidth', -1)
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType

In [3]:
#sc.stop()
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf
SparkContext.setSystemProperty('spark.executor.memory', '6g')
#SparkContext.setSystemProperty('spark.rdd.compress', True)
SparkContext.setSystemProperty('spark.driver.memory', '16g')
sc = SparkContext(master="local[4]")
print(sc)


<SparkContext master=local[4] appName=pyspark-shell>


In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LinearSVC,NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

In [5]:
train_path='/home/ucsddse230/work/final_project/post_azure/training_features.parquet'
test_path='/home/ucsddse230/work/final_project/post_azure/testing_features.parquet'
model_path='/home/ucsddse230/work/final_project/model'
dl_path='/home/ucsddse230/work/final_project/dl_model/'

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
train_data= spark.read.parquet(train_path)
test_data= spark.read.parquet(test_path)

## happy modeling
>please explore the the multilayer perceptron here and many others
https://spark.apache.org/docs/2.3.0/ml-classification-regression.html


In [8]:
train_data.columns

['HADM_ID', 'SUBJECT_ID', 'label', 'tokens_clean', 'features']

In [9]:
print(train_data.count(),len(train_data.columns))
print(test_data.count(),len(test_data.columns))

40795 5
10318 5


In [69]:
pos_count = train_data[train_data.label==1].count()
neg_count = train_data[train_data.label==0].count()

In [10]:
train_data.head()

Row(HADM_ID=100009, SUBJECT_ID=533, label=0, tokens_clean=['date', 'discharge', 'date', 'date', 'birth', 'sex', 'adverse', 'drug', 'complaint', 'angina', 'major', 'surgical', 'procedure', 'posterior', 'descending', 'artery', 'marginal', 'right', 'atrial', 'lead', 'history', 'present', 'illness', 'man', 'coronary', 'disease', 'ami', 'well', 'angina', 'exertion', 'angina', 'treatment', 'cardiac', 'vessel', 'disease', 'coronary', 'bypass', 'cardiac', 'date', 'place', 'chronic', 'total', 'occlusion', 'distal', 'chronic', 'total', 'occlusion', 'non', 'new', 'complex', 'stenosis', 'prox', 'large', 'old', 'patent', 'mod', 'systolic', 'anterior', 'apical', 'apical', 'reduced', 'past', 'medical', 'history', 'ami', 'cardiomyopathy', 'study', 'ventricular', 'tachycardia', 'atrial', 'flutter', 'hypertension', 'insulin', 'dependent', 'diabetes', 'mellitus', 'obesity', 'conduction', 'disease', 'peripheral', 'vascular', 'disease', 'right', 'fem', 'bypas', 'left', 'leg', 'right', 'thigh', 'tumor', 'ra

In [119]:
from utils import train, evaluate, display_metrics, classification_metrics

In [24]:
#Gradient Boosted Trees
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=100)
gbtModel = gbt.fit(train_data)
print(gbt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0)

In [25]:
#Train prediction
predictions =gbtModel.transform(train_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("GBT",Y_pred,Y_true)

______________________________________________
Classifier: GBT
Accuracy: 0.9423213629121215
AUC: 0.52939240281477
Precision: 0.9607843137254902
Recall: 0.058941459502806735
F1-score: 0.11106913486966377
______________________________________________



In [26]:
#Test prediction
predictions =gbtModel.transform(test_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("GBT",Y_pred,Y_true)

______________________________________________
Classifier: GBT
Accuracy: 0.9370032952122505
AUC: 0.5105605792307486
Precision: 0.4166666666666667
Recall: 0.023291925465838508
F1-score: 0.044117647058823525
______________________________________________



In [ ]:
gbtModel.save(model_path)

In [84]:
#Logistic Regression
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_data)
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [85]:
#Train prediction
predictions =lrModel.transform(train_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("Logistic Regression",Y_pred,Y_true)

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.9495526412550558
AUC: 0.6271442365308498
Precision: 0.7534883720930232
Recall: 0.2598235765838011
F1-score: 0.38640429338103754
______________________________________________



In [86]:
#Test prediction
predictions =lrModel.transform(test_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("Logistic Regression",Y_pred,Y_true)

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.9147121535181236
AUC: 0.5233132735885521
Precision: 0.1467065868263473
Recall: 0.07608695652173914
F1-score: 0.10020449897750511
______________________________________________



In [99]:
#SVC
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(maxIter=100)
svcModel = lsvc.fit(train_data)
print(lsvc.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 100)
predictionCol: prediction column name. (default: prediction)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: The threshold in binary classification applied to the linear model prediction.  This threshold can be any real number, where Inf will make all predictions 0.0 and -Inf will make all predictions 1.0. (default: 0.0, current: 0.1)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empt

In [100]:
#Train prediction
predictions = svcModel.transform(train_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("SVM",Y_pred,Y_true)

______________________________________________
Classifier: SVM
Accuracy: 0.9405564407402868
AUC: 0.5147703330058928
Precision: 0.9367088607594937
Recall: 0.02967121090617482
F1-score: 0.05752040419743489
______________________________________________



In [101]:
#Test prediction
predictions = svcModel.transform(test_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("SVM",Y_pred,Y_true)

______________________________________________
Classifier: SVM
Accuracy: 0.9358402791238613
AUC: 0.5019685216312663
Precision: 0.15384615384615385
Recall: 0.006211180124223602
F1-score: 0.011940298507462685
______________________________________________



In [92]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",thresholds=[neg_count,pos_count])

# train the model
nbModel = nb.fit(train_data)

In [93]:
print(nb.explainParams())

featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
modelType: The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli. (default: multinomial, current: multinomial)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
smoothing: The smoothing parameter, should be >= 0, default is 1.0 (default: 1.0, current: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest

In [72]:
#Train metrics
predictions = nbModel.transform(train_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("Naive Bayes",Y_pred,Y_true)

______________________________________________
Classifier: Naive Bayes
Accuracy: 0.6391959798994975
AUC: 0.637667557826625
Precision: 0.10300707930116257
Recall: 0.6359262229350441
F1-score: 0.1772958470739478
______________________________________________



In [73]:
#Test prediction
predictions = nbModel.transform(test_data)
Y_true=np.array(predictions.select('label').collect())
Y_pred=np.array(predictions.select('prediction').collect())
display_metrics("Naive Bayes",Y_pred,Y_true)

______________________________________________
Classifier: Naive Bayes
Accuracy: 0.6355882923047103
AUC: 0.6128911200798195
Precision: 0.09762396694214875
Recall: 0.5869565217391305
F1-score: 0.16740478299379982
______________________________________________



In [102]:
#Uncomment below line to install torch
#!pip install torch

In [109]:
#DL libraries and pandas and numpy
import numpy as np
import pandas as pd
from scipy import sparse
import torch
from torch.utils.data import TensorDataset, Dataset


In [104]:
#Convert to Pandas dataframe
train_df = train_data.toPandas()
test_df = test_data.toPandas()

In [105]:
print(train_df.shape,test_df.shape)

(40795, 5) (10318, 5)


In [106]:
train_y = np.array(train_df['label'].values)
test_y = np.array(test_df['label'].values)

In [158]:
#Create tensor datasets for MLP
#train dataset
target = torch.from_numpy(train_y.astype('long')).view(-1)

series = train_df['features'].apply(lambda x : np.array(x.toArray())).as_matrix().reshape(-1,1)
features = np.apply_along_axis(lambda x : x[0], 1, series)
train_dataset = TensorDataset(torch.from_numpy(features.astype('float32')), target)

#test dataset
target = torch.from_numpy(test_y.astype('long')).view(-1)

series = test_df['features'].apply(lambda x : np.array(x.toArray())).as_matrix().reshape(-1,1)
features = np.apply_along_axis(lambda x : x[0], 1, series)
test_dataset = TensorDataset(torch.from_numpy(features.astype('float32')), target)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

In [159]:
import torch.nn as nn
import torch.nn.functional as F

class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()
        self.hidden1 = nn.Linear(4499,2056)
        self.hidden2 = nn.Linear(2056,1028)
        self.hidden3 = nn.Linear(1028,512)
        self.hidden4 = nn.Linear(512,256)
        self.hidden5 = nn.Linear(256,64)
        self.hidden6 = nn.Linear(64,16)
        self.out = nn.Linear(16,2)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = F.relu(self.hidden5(x))
        x = F.relu(self.hidden6(x))
        x = self.out(x)
        return x

In [160]:
import torch.optim as optim
model = MyMLP()
save_file = 'MyMLP.pth'
class_weightage = torch.tensor([pos_count/neg_count,1-pos_count/neg_count])
print(class_weightage)
criterion = nn.CrossEntropyLoss(class_weightage)
optimizer = optim.Adam(model.parameters())

best_val_acc = 0.0
NUM_EPOCHS=3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tensor([0.0651, 0.9349])


In [161]:
best_auc = 0.0
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []
for epoch in range(NUM_EPOCHS):
    train_loss, train_accuracy = train(model, device, train_loader, criterion, optimizer, epoch)
    test_loss, test_accuracy, test_results = evaluate(model, device, test_loader, criterion, print_freq=1000)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    
    Y_true=np.array(test_results)[:,0]
    Y_pred=np.array(test_results)[:,1]
    acc, auc_, precision, recall, f1score = classification_metrics(Y_pred,Y_true)

    is_best = auc_ > best_auc  # let's keep the model that has the best AUC
    if is_best:
        print(auc_,best_auc)
        best_auc = auc_
        torch.save(model, (dl_path + save_file))


Epoch: [0][0/1275]	Time 0.237 (0.237)	Data 0.004 (0.004)	Loss 0.6593 (0.6593)	Accuracy 15.000 (15.000)
Epoch: [0][10/1275]	Time 0.132 (0.214)	Data 0.001 (0.001)	Loss 0.7355 (0.7165)	Accuracy 0.000 (5.455)
Epoch: [0][20/1275]	Time 0.119 (0.171)	Data 0.001 (0.001)	Loss 0.7111 (0.7080)	Accuracy 3.000 (6.571)
Epoch: [0][30/1275]	Time 0.121 (0.158)	Data 0.001 (0.001)	Loss 0.7106 (0.7044)	Accuracy 84.000 (20.065)
Epoch: [0][40/1275]	Time 0.117 (0.151)	Data 0.001 (0.001)	Loss 0.6939 (0.7029)	Accuracy 50.000 (33.488)
Epoch: [0][50/1275]	Time 0.116 (0.146)	Data 0.001 (0.001)	Loss 0.7667 (0.7048)	Accuracy 84.000 (43.725)
Epoch: [0][60/1275]	Time 0.165 (0.145)	Data 0.001 (0.001)	Loss 0.6956 (0.7033)	Accuracy 12.000 (45.180)
Epoch: [0][70/1275]	Time 0.458 (0.158)	Data 0.001 (0.001)	Loss 0.6909 (0.7026)	Accuracy 9.000 (39.789)
Epoch: [0][80/1275]	Time 0.140 (0.162)	Data 0.001 (0.001)	Loss 0.6822 (0.7003)	Accuracy 90.000 (41.531)
Epoch: [0][90/1275]	Time 0.134 (0.159)	Data 0.001 (0.001)	Loss 0.7577 

Epoch: [0][790/1275]	Time 0.264 (0.188)	Data 0.001 (0.001)	Loss 0.8064 (0.6887)	Accuracy 50.000 (78.217)
Epoch: [0][800/1275]	Time 0.249 (0.189)	Data 0.001 (0.001)	Loss 0.6344 (0.6880)	Accuracy 75.000 (78.084)
Epoch: [0][810/1275]	Time 0.238 (0.190)	Data 0.001 (0.001)	Loss 0.6186 (0.6877)	Accuracy 65.000 (77.942)
Epoch: [0][820/1275]	Time 0.244 (0.190)	Data 0.001 (0.001)	Loss 0.6583 (0.6878)	Accuracy 75.000 (78.049)
Epoch: [0][830/1275]	Time 0.237 (0.191)	Data 0.001 (0.001)	Loss 0.6028 (0.6871)	Accuracy 75.000 (78.019)
Epoch: [0][840/1275]	Time 0.239 (0.192)	Data 0.001 (0.001)	Loss 0.6270 (0.6869)	Accuracy 78.000 (77.951)
Epoch: [0][850/1275]	Time 0.242 (0.192)	Data 0.001 (0.001)	Loss 0.6936 (0.6868)	Accuracy 81.000 (77.938)
Epoch: [0][860/1275]	Time 0.254 (0.193)	Data 0.001 (0.001)	Loss 0.6348 (0.6857)	Accuracy 90.000 (78.046)
Epoch: [0][870/1275]	Time 0.266 (0.194)	Data 0.001 (0.001)	Loss 0.5779 (0.6851)	Accuracy 90.000 (78.185)
Epoch: [0][880/1275]	Time 0.246 (0.194)	Data 0.001 (0.0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type MyMLP. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0.5 0.0
Epoch: [1][0/1275]	Time 0.351 (0.351)	Data 0.003 (0.003)	Loss 0.7492 (0.7492)	Accuracy 87.000 (87.000)
Epoch: [1][10/1275]	Time 0.262 (0.278)	Data 0.001 (0.001)	Loss 0.6533 (0.6891)	Accuracy 96.000 (91.091)
Epoch: [1][20/1275]	Time 0.259 (0.273)	Data 0.001 (0.001)	Loss 0.7139 (0.6888)	Accuracy 78.000 (89.143)
Epoch: [1][30/1275]	Time 0.259 (0.270)	Data 0.001 (0.001)	Loss 0.6658 (0.6756)	Accuracy 62.000 (85.710)
Epoch: [1][40/1275]	Time 0.256 (0.269)	Data 0.001 (0.001)	Loss 0.6658 (0.6742)	Accuracy 96.000 (85.537)
Epoch: [1][50/1275]	Time 0.260 (0.268)	Data 0.001 (0.001)	Loss 0.6455 (0.6722)	Accuracy 81.000 (85.765)
Epoch: [1][60/1275]	Time 0.265 (0.268)	Data 0.001 (0.001)	Loss 0.6059 (0.6742)	Accuracy 71.000 (84.180)
Epoch: [1][70/1275]	Time 0.261 (0.267)	Data 0.001 (0.001)	Loss 0.4474 (0.6630)	Accuracy 93.000 (84.042)
Epoch: [1][80/1275]	Time 0.262 (0.266)	Data 0.001 (0.001)	Loss 0.6919 (0.6682)	Accuracy 71.000 (84.210)
Epoch: [1][90/1275]	Time 0.263 (0.266)	Data 0.001 (0.001)

Epoch: [1][790/1275]	Time 0.265 (0.271)	Data 0.001 (0.001)	Loss 0.8209 (0.6478)	Accuracy 65.000 (76.920)
Epoch: [1][800/1275]	Time 0.266 (0.271)	Data 0.001 (0.001)	Loss 0.6182 (0.6478)	Accuracy 90.000 (77.046)
Epoch: [1][810/1275]	Time 0.267 (0.271)	Data 0.001 (0.001)	Loss 0.5686 (0.6474)	Accuracy 96.000 (77.217)
Epoch: [1][820/1275]	Time 0.262 (0.271)	Data 0.001 (0.001)	Loss 0.5463 (0.6517)	Accuracy 90.000 (77.359)
Epoch: [1][830/1275]	Time 0.265 (0.271)	Data 0.001 (0.001)	Loss 0.7307 (0.6516)	Accuracy 75.000 (77.416)
Epoch: [1][840/1275]	Time 0.262 (0.271)	Data 0.001 (0.001)	Loss 0.6912 (0.6516)	Accuracy 62.000 (77.290)
Epoch: [1][850/1275]	Time 0.264 (0.271)	Data 0.001 (0.001)	Loss 0.5952 (0.6516)	Accuracy 81.000 (77.231)
Epoch: [1][860/1275]	Time 0.262 (0.271)	Data 0.001 (0.001)	Loss 0.6022 (0.6512)	Accuracy 71.000 (77.207)
Epoch: [1][870/1275]	Time 0.264 (0.271)	Data 0.001 (0.001)	Loss 0.7179 (0.6511)	Accuracy 78.000 (77.225)
Epoch: [1][880/1275]	Time 0.263 (0.271)	Data 0.001 (0.0

Epoch: [2][280/1275]	Time 0.297 (0.289)	Data 0.001 (0.001)	Loss 0.6693 (0.6314)	Accuracy 90.000 (80.854)
Epoch: [2][290/1275]	Time 0.293 (0.290)	Data 0.001 (0.001)	Loss 0.5587 (0.6321)	Accuracy 75.000 (80.825)
Epoch: [2][300/1275]	Time 0.298 (0.290)	Data 0.001 (0.001)	Loss 0.6473 (0.6318)	Accuracy 84.000 (80.900)
Epoch: [2][310/1275]	Time 0.299 (0.291)	Data 0.001 (0.001)	Loss 0.5638 (0.6316)	Accuracy 78.000 (80.675)
Epoch: [2][320/1275]	Time 0.297 (0.291)	Data 0.001 (0.001)	Loss 0.5574 (0.6313)	Accuracy 93.000 (80.639)
Epoch: [2][330/1275]	Time 0.295 (0.291)	Data 0.001 (0.001)	Loss 0.4936 (0.6302)	Accuracy 84.000 (80.719)
Epoch: [2][340/1275]	Time 0.305 (0.292)	Data 0.001 (0.001)	Loss 0.6434 (0.6295)	Accuracy 59.000 (80.522)
Epoch: [2][350/1275]	Time 0.299 (0.292)	Data 0.001 (0.001)	Loss 0.8328 (0.6293)	Accuracy 56.000 (80.194)
Epoch: [2][360/1275]	Time 0.293 (0.293)	Data 0.001 (0.001)	Loss 0.5923 (0.6287)	Accuracy 87.000 (79.972)
Epoch: [2][370/1275]	Time 0.297 (0.294)	Data 0.001 (0.0

Epoch: [2][1060/1275]	Time 0.316 (0.311)	Data 0.001 (0.001)	Loss 0.6745 (0.6223)	Accuracy 78.000 (79.935)
Epoch: [2][1070/1275]	Time 0.327 (0.311)	Data 0.001 (0.001)	Loss 0.5949 (0.6221)	Accuracy 65.000 (79.838)
Epoch: [2][1080/1275]	Time 0.322 (0.311)	Data 0.001 (0.001)	Loss 0.6364 (0.6214)	Accuracy 87.000 (79.844)
Epoch: [2][1090/1275]	Time 0.322 (0.311)	Data 0.001 (0.001)	Loss 0.4512 (0.6213)	Accuracy 90.000 (79.909)
Epoch: [2][1100/1275]	Time 0.320 (0.311)	Data 0.001 (0.001)	Loss 0.5821 (0.6208)	Accuracy 68.000 (79.951)
Epoch: [2][1110/1275]	Time 0.315 (0.312)	Data 0.001 (0.001)	Loss 0.6156 (0.6206)	Accuracy 62.000 (79.872)
Epoch: [2][1120/1275]	Time 0.317 (0.312)	Data 0.001 (0.001)	Loss 0.5298 (0.6207)	Accuracy 87.000 (79.812)
Epoch: [2][1130/1275]	Time 0.322 (0.312)	Data 0.001 (0.001)	Loss 0.5038 (0.6199)	Accuracy 90.000 (79.853)
Epoch: [2][1140/1275]	Time 0.312 (0.312)	Data 0.001 (0.001)	Loss 0.6910 (0.6199)	Accuracy 81.000 (79.855)
Epoch: [2][1150/1275]	Time 0.316 (0.312)	Data 

In [162]:
#Load model
best_model = torch.load(dl_path + save_file)

In [163]:
#Train metrics
train_loss, train_accuracy, train_results = evaluate(best_model, device, train_loader, criterion, print_freq=1000)

Test: [0/1275]	Time 0.034 (0.034)	Loss 0.6599 (0.6599)	Accuracy 81.000 (81.000)
Test: [1000/1275]	Time 0.022 (0.019)	Loss 0.6083 (0.6213)	Accuracy 78.000 (80.417)


In [164]:
Y_true=np.array(train_results)[:,0]
Y_pred=np.array(train_results)[:,1]
display_metrics("MLP",Y_pred,Y_true)

______________________________________________
Classifier: MLP
Accuracy: 0.8091923029783061
AUC: 0.6657873625297879
Precision: 0.1607234479220113
Recall: 0.5024057738572574
F1-score: 0.24353741496598638
______________________________________________



In [165]:
#Test metrics
test_loss, test_accuracy, test_results = evaluate(best_model, device, test_loader, criterion, print_freq=900000)

Test: [0/323]	Time 0.031 (0.031)	Loss 0.6925 (0.6925)	Accuracy 75.000 (75.000)


In [166]:
Y_true=np.array(test_results)[:,0]
Y_pred=np.array(test_results)[:,1]
display_metrics("MLP",Y_pred,Y_true)

______________________________________________
Classifier: MLP
Accuracy: 0.7978290366350068
AUC: 0.6204180187144386
Precision: 0.13585858585858585
Recall: 0.41770186335403725
F1-score: 0.205030487804878
______________________________________________



In [167]:
#Create tensor datasets for CNN
#train dataset
target = torch.from_numpy(train_y.astype('long'))

series = train_df['features'].apply(lambda x : np.array(x.toArray())).as_matrix().reshape(-1,1)
features = np.apply_along_axis(lambda x : x[0], 1, series)
train_dataset = TensorDataset(torch.from_numpy(features.astype('float32')).unsqueeze(1), target)

#test dataset
target = torch.from_numpy(test_y.astype('long'))

series = test_df['features'].apply(lambda x : np.array(x.toArray())).as_matrix().reshape(-1,1)
features = np.apply_along_axis(lambda x : x[0], 1, series)
test_dataset = TensorDataset(torch.from_numpy(features.astype('float32')).unsqueeze(1), target)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

In [176]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=1121, out_features=64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
#m1: [128 x 4484], m2: [128 x 64]
    def forward(self, x):
#torch.Size([32, 16, 1121])
#torch.Size([32, 1121])
#torch.Size([32, 17936])

        x = self.pool(torch.nn.functional.relu(self.conv1(x)))        
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        
        #flatten
        x = x[:,0]        
        x = F.relu(self.fc1(x))        
        x = F.dropout(x, training=self.training)        
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [177]:
model = MyCNN()
save_file = 'MyCNN.pth'
class_weightage = torch.tensor([pos_count/neg_count,1-pos_count/neg_count])
print(class_weightage)
criterion = nn.CrossEntropyLoss(class_weightage)
optimizer = optim.Adam(model.parameters())

best_val_acc = 0.0
NUM_EPOCHS=3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tensor([0.0651, 0.9349])


In [178]:
best_auc = 0.0
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []
for epoch in range(NUM_EPOCHS):
    train_loss, train_accuracy = train(model, device, train_loader, criterion, optimizer, epoch)
    test_loss, test_accuracy, test_results = evaluate(model, device, test_loader, criterion, print_freq=1000)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    
    Y_true=np.array(test_results)[:,0]
    Y_pred=np.array(test_results)[:,1]
    acc, auc_, precision, recall, f1score = classification_metrics(Y_pred,Y_true)

    is_best = auc_ > best_auc  # let's keep the model that has the best AUC
    if is_best:
        print(auc_,best_auc)
        best_auc = auc_
        torch.save(model, (dl_path + save_file))


Epoch: [0][0/1275]	Time 0.070 (0.070)	Data 0.003 (0.003)	Loss 0.7876 (0.7876)	Accuracy 0.000 (0.000)
Epoch: [0][10/1275]	Time 0.044 (0.054)	Data 0.001 (0.001)	Loss 0.7176 (0.7147)	Accuracy 3.000 (4.909)
Epoch: [0][20/1275]	Time 0.043 (0.050)	Data 0.001 (0.001)	Loss 0.6871 (0.7109)	Accuracy 65.000 (11.762)
Epoch: [0][30/1275]	Time 0.045 (0.049)	Data 0.001 (0.001)	Loss 0.6354 (0.7041)	Accuracy 96.000 (33.323)
Epoch: [0][40/1275]	Time 0.062 (0.050)	Data 0.001 (0.001)	Loss 0.7436 (0.7103)	Accuracy 71.000 (44.146)
Epoch: [0][50/1275]	Time 0.043 (0.049)	Data 0.001 (0.001)	Loss 0.7086 (0.7067)	Accuracy 46.000 (45.804)
Epoch: [0][60/1275]	Time 0.044 (0.049)	Data 0.001 (0.001)	Loss 0.7101 (0.7054)	Accuracy 71.000 (48.721)
Epoch: [0][70/1275]	Time 0.044 (0.049)	Data 0.001 (0.001)	Loss 0.6465 (0.7016)	Accuracy 84.000 (52.507)
Epoch: [0][80/1275]	Time 0.044 (0.049)	Data 0.001 (0.001)	Loss 0.6166 (0.7008)	Accuracy 100.000 (56.815)
Epoch: [0][90/1275]	Time 0.040 (0.049)	Data 0.001 (0.001)	Loss 0.642

Epoch: [0][790/1275]	Time 0.053 (0.048)	Data 0.001 (0.001)	Loss 0.5560 (0.6870)	Accuracy 100.000 (89.350)
Epoch: [0][800/1275]	Time 0.041 (0.048)	Data 0.001 (0.001)	Loss 0.5966 (0.6864)	Accuracy 96.000 (89.423)
Epoch: [0][810/1275]	Time 0.042 (0.048)	Data 0.001 (0.001)	Loss 0.6124 (0.6864)	Accuracy 96.000 (89.465)
Epoch: [0][820/1275]	Time 0.048 (0.048)	Data 0.001 (0.001)	Loss 0.6323 (0.6864)	Accuracy 96.000 (89.515)
Epoch: [0][830/1275]	Time 0.047 (0.048)	Data 0.001 (0.001)	Loss 0.5257 (0.6858)	Accuracy 100.000 (89.578)
Epoch: [0][840/1275]	Time 0.044 (0.048)	Data 0.001 (0.001)	Loss 0.6325 (0.6862)	Accuracy 96.000 (89.615)
Epoch: [0][850/1275]	Time 0.044 (0.048)	Data 0.001 (0.001)	Loss 0.6463 (0.6865)	Accuracy 96.000 (89.644)
Epoch: [0][860/1275]	Time 0.044 (0.048)	Data 0.001 (0.001)	Loss 0.6930 (0.6863)	Accuracy 90.000 (89.688)
Epoch: [0][870/1275]	Time 0.041 (0.048)	Data 0.001 (0.001)	Loss 0.6648 (0.6864)	Accuracy 96.000 (89.713)
Epoch: [0][880/1275]	Time 0.041 (0.048)	Data 0.001 (0

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: [1][10/1275]	Time 0.041 (0.047)	Data 0.001 (0.001)	Loss 0.6079 (0.6397)	Accuracy 75.000 (70.273)
Epoch: [1][20/1275]	Time 0.048 (0.048)	Data 0.001 (0.001)	Loss 0.4996 (0.6332)	Accuracy 81.000 (72.476)
Epoch: [1][30/1275]	Time 0.041 (0.048)	Data 0.001 (0.001)	Loss 0.5219 (0.6191)	Accuracy 84.000 (77.065)
Epoch: [1][40/1275]	Time 0.041 (0.047)	Data 0.001 (0.001)	Loss 0.5954 (0.6165)	Accuracy 90.000 (78.829)
Epoch: [1][50/1275]	Time 0.049 (0.048)	Data 0.001 (0.001)	Loss 0.6822 (0.6178)	Accuracy 84.000 (80.373)
Epoch: [1][60/1275]	Time 0.042 (0.048)	Data 0.001 (0.001)	Loss 0.8050 (0.6265)	Accuracy 68.000 (81.049)
Epoch: [1][70/1275]	Time 0.046 (0.048)	Data 0.001 (0.001)	Loss 0.6150 (0.6187)	Accuracy 96.000 (82.099)
Epoch: [1][80/1275]	Time 0.050 (0.047)	Data 0.001 (0.001)	Loss 0.7411 (0.6313)	Accuracy 90.000 (82.173)
Epoch: [1][90/1275]	Time 0.052 (0.048)	Data 0.001 (0.001)	Loss 0.8348 (0.6369)	Accuracy 87.000 (82.231)
Epoch: [1][100/1275]	Time 0.045 (0.048)	Data 0.001 (0.001)	Loss 

Epoch: [1][800/1275]	Time 0.042 (0.052)	Data 0.001 (0.001)	Loss 0.7497 (0.6517)	Accuracy 84.000 (80.139)
Epoch: [1][810/1275]	Time 0.042 (0.051)	Data 0.001 (0.001)	Loss 0.5236 (0.6513)	Accuracy 84.000 (80.231)
Epoch: [1][820/1275]	Time 0.045 (0.051)	Data 0.001 (0.001)	Loss 0.4702 (0.6508)	Accuracy 93.000 (80.335)
Epoch: [1][830/1275]	Time 0.042 (0.051)	Data 0.001 (0.001)	Loss 0.7587 (0.6504)	Accuracy 87.000 (80.430)
Epoch: [1][840/1275]	Time 0.044 (0.051)	Data 0.001 (0.001)	Loss 0.6709 (0.6506)	Accuracy 81.000 (80.501)
Epoch: [1][850/1275]	Time 0.044 (0.051)	Data 0.001 (0.001)	Loss 0.5219 (0.6507)	Accuracy 84.000 (80.491)
Epoch: [1][860/1275]	Time 0.045 (0.051)	Data 0.001 (0.001)	Loss 0.5969 (0.6509)	Accuracy 78.000 (80.424)
Epoch: [1][870/1275]	Time 0.064 (0.051)	Data 0.001 (0.001)	Loss 0.7219 (0.6514)	Accuracy 65.000 (80.332)
Epoch: [1][880/1275]	Time 0.058 (0.051)	Data 0.001 (0.001)	Loss 0.7335 (0.6512)	Accuracy 84.000 (80.321)
Epoch: [1][890/1275]	Time 0.045 (0.051)	Data 0.001 (0.0

Epoch: [2][290/1275]	Time 0.051 (0.054)	Data 0.001 (0.001)	Loss 0.6885 (0.6355)	Accuracy 78.000 (77.670)
Epoch: [2][300/1275]	Time 0.041 (0.054)	Data 0.001 (0.001)	Loss 0.4781 (0.6340)	Accuracy 87.000 (77.870)
Epoch: [2][310/1275]	Time 0.062 (0.054)	Data 0.001 (0.001)	Loss 0.6818 (0.6356)	Accuracy 84.000 (77.958)
Epoch: [2][320/1275]	Time 0.042 (0.054)	Data 0.001 (0.001)	Loss 0.4960 (0.6362)	Accuracy 81.000 (77.903)
Epoch: [2][330/1275]	Time 0.045 (0.054)	Data 0.001 (0.001)	Loss 0.7416 (0.6350)	Accuracy 71.000 (77.813)
Epoch: [2][340/1275]	Time 0.041 (0.053)	Data 0.001 (0.001)	Loss 0.7741 (0.6357)	Accuracy 78.000 (77.833)
Epoch: [2][350/1275]	Time 0.041 (0.053)	Data 0.001 (0.001)	Loss 0.7000 (0.6364)	Accuracy 90.000 (77.792)
Epoch: [2][360/1275]	Time 0.045 (0.053)	Data 0.001 (0.001)	Loss 0.8334 (0.6357)	Accuracy 65.000 (77.765)
Epoch: [2][370/1275]	Time 0.051 (0.053)	Data 0.001 (0.001)	Loss 0.4360 (0.6350)	Accuracy 87.000 (77.747)
Epoch: [2][380/1275]	Time 0.059 (0.053)	Data 0.003 (0.0

Epoch: [2][1070/1275]	Time 0.043 (0.053)	Data 0.001 (0.001)	Loss 0.6723 (0.6356)	Accuracy 71.000 (75.948)
Epoch: [2][1080/1275]	Time 0.041 (0.053)	Data 0.001 (0.001)	Loss 0.6297 (0.6363)	Accuracy 71.000 (75.856)
Epoch: [2][1090/1275]	Time 0.045 (0.053)	Data 0.001 (0.001)	Loss 0.5647 (0.6370)	Accuracy 68.000 (75.736)
Epoch: [2][1100/1275]	Time 0.045 (0.053)	Data 0.001 (0.001)	Loss 0.6050 (0.6372)	Accuracy 78.000 (75.657)
Epoch: [2][1110/1275]	Time 0.046 (0.053)	Data 0.001 (0.001)	Loss 0.6681 (0.6373)	Accuracy 53.000 (75.563)
Epoch: [2][1120/1275]	Time 0.042 (0.053)	Data 0.001 (0.001)	Loss 0.6050 (0.6373)	Accuracy 75.000 (75.478)
Epoch: [2][1130/1275]	Time 0.044 (0.053)	Data 0.001 (0.001)	Loss 0.5980 (0.6373)	Accuracy 65.000 (75.455)
Epoch: [2][1140/1275]	Time 0.044 (0.052)	Data 0.001 (0.001)	Loss 0.7893 (0.6375)	Accuracy 81.000 (75.443)
Epoch: [2][1150/1275]	Time 0.042 (0.052)	Data 0.001 (0.001)	Loss 0.5876 (0.6371)	Accuracy 71.000 (75.447)
Epoch: [2][1160/1275]	Time 0.045 (0.052)	Data 

In [179]:
#Load model
best_model = torch.load(dl_path + save_file)

In [180]:
#Train metrics
train_loss, train_accuracy, train_results = evaluate(best_model, device, train_loader, criterion, print_freq=1000)

Test: [0/1275]	Time 0.032 (0.032)	Loss 0.6760 (0.6760)	Accuracy 56.000 (56.000)
Test: [1000/1275]	Time 0.016 (0.018)	Loss 0.6539 (0.6291)	Accuracy 68.000 (72.607)


In [181]:
Y_true=np.array(train_results)[:,0]
Y_pred=np.array(train_results)[:,1]
display_metrics("CNN",Y_pred,Y_true)

______________________________________________
Classifier: CNN
Accuracy: 0.7302120357886996
AUC: 0.6441553040788401
Precision: 0.12121751512993947
Recall: 0.5461106655974338
F1-score: 0.19839766933721778
______________________________________________



In [182]:
#Test metrics
test_loss, test_accuracy, test_results = evaluate(best_model, device, test_loader, criterion, print_freq=900000)

Test: [0/323]	Time 0.026 (0.026)	Loss 0.7479 (0.7479)	Accuracy 71.000 (71.000)


In [183]:
Y_true=np.array(test_results)[:,0]
Y_pred=np.array(test_results)[:,1]
display_metrics("CNN",Y_pred,Y_true)

______________________________________________
Classifier: CNN
Accuracy: 0.721554564838147
AUC: 0.6051069203872325
Precision: 0.10715544589354953
Recall: 0.4720496894409938
F1-score: 0.17466245331801206
______________________________________________

